# "Fallecidos DEIS por comuna"
> Fallecidos DEIS por comuna.

- toc: true 
- badges: true
- comments: true
- author: Alonso Silva Allende
- categories: [jupyter]
- image: images/diagram.png

In [1]:
#hide
import numpy as np
import pandas as pd
import altair as alt
from IPython.display import display, Markdown, display_html, HTML

In [2]:
#hide
update_date = pd.to_datetime('today') - pd.offsets.Hour(19)

In [3]:
#hide_input
display(Markdown(f"Última actualización: {update_date.strftime('%d/%m/%Y')}."))

Última actualización: 02/08/2020.

# Defunciones diarias causa COVID-19 (COVID-19 Confirmado + COVID-19 Sospechoso) por comuna de la Región Metropolitana

In [4]:
#hide
data_raw = pd.read_csv(
    "/Users/alonsosilva/COVID-19/DEFUNCIONES_FUENTE_DEIS_2016_2020_30072020.csv", encoding='ISO-8859-1', sep=";")

In [5]:
#hide
data_raw = data_raw.T.reset_index().T

In [6]:
#hide
data_raw = data_raw.rename(
    columns={0:"año",
             1:"fecha",
             2:"género", 
             3:"edad", 
             4:"comuna_código", 
             5:"comuna", 
             6:"región", 
             7:"código_detalle",
             8:"causa_detalle",
             9:"código",
            10:"causa"})

In [7]:
#hide
data_RM = data_raw.query("región=='Metropolitana de Santiago' & código == 'U07'")

In [8]:
#hide
len(data_RM)

10014

In [9]:
#hide
last_date = pd.to_datetime(data_RM['fecha']).sort_values().iloc[-1].strftime('%Y-%m-%d')
first_date = pd.to_datetime(data_RM['fecha']).sort_values().iloc[0].strftime('%Y-%m-%d')
first_date, last_date

('2020-03-15', '2020-07-29')

In [10]:
#hide
county_deaths_RM = pd.DataFrame()
county_deaths_RM["Fecha"] = pd.date_range(start=first_date, end=last_date)
county_deaths_RM = county_deaths_RM.set_index("Fecha")

In [11]:
#hide
for comuna in data_RM["comuna"].unique():
    aux = data_RM[data_RM["comuna"] == f"{comuna}"]
    deaths = pd.Series(dtype='float64')
    for date in aux["fecha"].unique():
        deaths[f'{date}'] = len(aux.query(f"fecha == '{date}'"))
    deaths.index = pd.to_datetime(deaths.index)
    county_deaths_RM[f'{comuna}'] = deaths

In [12]:
#hide
county_deaths_RM = county_deaths_RM.T.sort_index().T

In [13]:
#hide
data_long_form = county_deaths_RM.reset_index().melt("Fecha", var_name="Comuna", value_name = "Fallecidos confirmados")

In [14]:
#hide
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [15]:
#hide_input
input_dropdown = alt.binding_select(options=data_long_form['Comuna'].unique())
selection1 = alt.selection_single(fields=['Comuna'], bind=input_dropdown, name=' ')
selection2 = alt.selection_multi(fields=['Comuna'], on='mouseover')

color = alt.condition(selection1 | selection2,
                    alt.Color('Comuna:N', scale=alt.Scale(scheme='category20'), legend=None),
                    alt.value('lightgray'))

bars1 = alt.Chart(data_long_form).mark_bar().encode(
    x=alt.X("Fecha", axis=alt.Axis(title="", domain=False, tickSize=0)),
    y=alt.Y("Fallecidos confirmados", axis=alt.Axis(title="número de defunciones")),
    tooltip = ["Fecha", "Comuna", "Fallecidos confirmados"],
    color = color
).properties(
    title=f"Nuevas defunciones causa COVID-19 (COVID-19 Confirmado + COVID-19 Sospechoso) por comuna de la R.M. al {pd.to_datetime(last_date).strftime('%d/%m')}",
    height=500,
    width=700).add_selection(
    selection1, selection2
).transform_filter(
    selection1
)

bars1

alt.Chart(...)

La disminución de defunciones en las últimas fechas es artificial y tiene que ver con los períodos de investigación del DEIS.

# Total defunciones causa COVID-19 (COVID-19 Confirmado + COVID-19 Sospechoso) por comuna de la Región Metropolitana

In [16]:
#hide
cumulative_county_deaths = county_deaths_RM.fillna(0).cumsum()

In [17]:
#hide
data_long_form = cumulative_county_deaths.reset_index().melt("Fecha", var_name="Comuna", value_name = "Fallecidos confirmados")

In [18]:
#hide_input
input_dropdown = alt.binding_select(options=data_long_form['Comuna'].unique())
selection1 = alt.selection_single(fields=['Comuna'], bind=input_dropdown, name=' ')
selection2 = alt.selection_multi(fields=['Comuna'], on='mouseover')

color = alt.condition(selection1 | selection2,
                    alt.Color('Comuna:N', scale=alt.Scale(scheme='category20'), legend=None),
                    alt.value('lightgray'))

bars1 = alt.Chart(data_long_form).mark_line().encode(
    x=alt.X("Fecha", axis=alt.Axis(title="", domain=False, tickSize=0)),
    y=alt.Y("Fallecidos confirmados", axis=alt.Axis(title="número de defunciones")),
    tooltip = ["Fecha", "Comuna", "Fallecidos confirmados"],
    color = color
).properties(
    title=f"Total defunciones causa COVID-19 (COVID-19 Confirmado + COVID-19 Sospechoso) por comuna de la R.M. al {pd.to_datetime(last_date).strftime('%d/%m')}",
    height=500,
    width=700).add_selection(
    selection1, selection2
).transform_filter(
    selection1
)

bars1

alt.Chart(...)

La disminución de defunciones en las últimas fechas es artificial y tiene que ver con los períodos de investigación del DEIS.

# Defunciones causa COVID-19 por comuna

In [19]:
#hide
confirmed_deaths = pd.Series(dtype="float64")
suspected_deaths = pd.Series(dtype='float64')
for comuna in data_RM["comuna"].unique():
    confirmed_deaths[f"{comuna}"] = len(data_raw[(data_raw["código_detalle"] == 'U07.1') & (data_raw["comuna"] == f'{comuna}')])
    suspected_deaths[f"{comuna}"] = len(data_raw[(data_raw["código_detalle"] == 'U07.2') & (data_raw["comuna"] == f'{comuna}')])

In [20]:
#hide
deaths = pd.DataFrame()
deaths["Comuna"] = data_RM["comuna"].unique()
deaths = deaths.set_index("Comuna")

In [21]:
#hide
deaths["COVID Confirmado"] = confirmed_deaths
deaths["COVID Sospechoso"] = suspected_deaths

In [22]:
#hide
deaths_long_format = deaths.reset_index().melt("Comuna", var_name="Causa", value_name="Fallecidos")

In [23]:
#hide_input
bars = alt.Chart(deaths_long_format).mark_bar(opacity=0.7, size=25).encode(
    x = alt.X('Fallecidos:Q', stack='zero'),
    y = alt.Y('Comuna', sort='-x'),
    tooltip = ['Comuna', 'Fallecidos'],
    color = alt.Color('Causa', sort=['Confirmado','Sospechoso']),
    order = alt.Order('Causa')
)

text = alt.Chart(deaths_long_format).mark_text(dx=-11, dy=0, color="#636363").encode(
     x=alt.X('Fallecidos:Q', stack='zero'),
     y=alt.Y('Comuna', sort='-x'),
     color=alt.Color('Causa', sort=['Confirmado','Sospechoso']),
     order = alt.Order("Causa"),
     text=alt.Text("Fallecidos", format=',.0f')
)
(bars+text).properties(
    title = f"Total fallecidos causa COVID por comuna de la R.M. al {pd.to_datetime(last_date).strftime('%d/%m')}",
    width = 600,
    height = alt.Step(30)
)

alt.LayerChart(...)

# Total fallecidos causa COVID por 100.000 habitantes por comuna de la Región Metropolitana

In [24]:
#hide
county_deaths = pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto38/CasosFallecidosPorComuna.csv")

In [25]:
#hide
county_deaths_RM = county_deaths.query(
    "Region=='Metropolitana'").drop(
    columns=["Region", "Codigo region", "Codigo comuna"]).set_index("Comuna")

In [26]:
#hide
county_deaths_RM = county_deaths_RM.T.rename(columns={"Penalolen":'Peñalolén','Alhue':'Alhué','Nunoa':'Ñuñoa','San Joaquin':'San Joaquín','San Ramon':'San Ramón','Conchali':'Conchalí','San Ramon':'San Ramón','Maipu':'Maipú','Curacavi':'Curacaví','Estacion Central':'Estación Central','Penaflor':'Peñaflor','Maria Pinto':'María Pinto','San Jose de Maipo':'San José de Maipo'}).T

In [27]:
#hide
deaths["Tasa de mortalidad COVID Confirmado"] = \
np.round(100000*deaths["COVID Confirmado"]/county_deaths_RM["Poblacion"], decimals=1)

In [28]:
#hide
deaths["Tasa de mortalidad COVID Sospechoso"] = \
np.round(100000*deaths["COVID Sospechoso"]/county_deaths_RM["Poblacion"], decimals=1)

In [29]:
#hide
deaths_long_format = deaths[["Tasa de mortalidad COVID Confirmado","Tasa de mortalidad COVID Sospechoso"]].rename(columns={"Tasa de mortalidad COVID Confirmado":"COVID Confirmado","Tasa de mortalidad COVID Sospechoso":"COVID Sospechoso"}).reset_index().melt("Comuna", var_name="Causa", value_name="Fallecidos")

In [30]:
#hide_input
bars = alt.Chart(deaths_long_format).mark_bar(opacity=0.7, size=25).encode(
    x = alt.X('Fallecidos:Q', stack='zero', axis=alt.Axis(title="Tasa de mortalidad")),
    y = alt.Y('Comuna', sort='-x'),
    tooltip = ['Comuna', 'Fallecidos'],
    color = alt.Color('Causa', sort=['Confirmado','Sospechoso']),
    order = alt.Order('Causa')
)

text = alt.Chart(deaths_long_format).mark_text(dx=-16, dy=0, color="#636363").encode(
     x=alt.X('Fallecidos:Q', stack='zero'),
     y=alt.Y('Comuna', sort='-x'),
     color=alt.Color('Causa', sort=['Confirmado','Sospechoso']),
     order = alt.Order("Causa"),
     text=alt.Text("Fallecidos", format=',.1f')
)
(bars+text).properties(
    title = f"Tasa de mortalidad causa COVID-19 por comuna de la R.M. al {pd.to_datetime(last_date).strftime('%d/%m')}",
    width = 600,
    height = alt.Step(30)
)

alt.LayerChart(...)